<h1>LycomO360_ex01_std_vs_trbchgd</h1>
<h2>notebook: rev004</h2>
<p>compatability with OpenModelica v1.17</p>
<h2>memo: </h2>
<p>compare Lycom O360 engine with turbochaged one</p>

<h1>Load modules </h1>

In [1]:
#****************************************
#    User must NOT edit code below 
#****************************************
import OMPython
from OMPython import OMCSessionZMQ
from OMPython import ModelicaSystem

import os
from os.path import expanduser

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import pathlib

from IPython.display import display, HTML
print('------------------------------')

------------------------------


In [26]:
#****************************************
#    User must NOT edit code below 
#****************************************
# ----- prepare for post process -----
n_df=500
df=[[]]*n_df
i_df= -1
print('----------')

----------


<h1>===========================================================================</h1>

<h1>Run simulation model #1</h1>

<h1>User settings: Library and Model information</h1>
<p>Edit the strings below </p>

In [3]:
dirWorkRelToHome='/OMPythonWork'

#dirModelRelToLibDir='/Examples/ThermodynamicCycles'
libs2load=["PropulsionSystem", "FluidSystemComponents", "WalkingInWorldOfThermoFluid", "AircraftDynamics", "SystemModels", "InteractiveSimulation"]
fullpathModel='PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01'
nmModel='LycomO360_ex01'
stopTime=90.0
outputFormat='csv'
print('------------------------------')

------------------------------


<h1>Setup </h1>

In [4]:
#**************************************************
# ----- setup -----
#**************************************************
#****************************************
#    User must NOT edit code below 
#****************************************
#*** get directory of this notebook
sys.path.append('..')
pathNotebook= os.getcwd()
print('------------------------------'); print('')

#*** distinguish OS
if(os.name=='nt'):
    dirHome=expanduser("~")
    typeOS='Windows (nt)'
elif(os.name=='posix'):
    dirHome=os.environ['HOME']
    typeOS= 'Linux or Mac (posix)'
##### end if #####

#*** direcotory of temporary work
dirWork= dirHome + dirWorkRelToHome

#*** get library directory
dirLib=''
sys.path.append('..')
dirLib= pathlib.Path('../../')
dirLib_upper1= pathlib.Path('../../')

dirLib= dirLib.resolve()
dirLib= str(dirLib)
dirLib= dirLib.replace('\\', '/')

dirLib_upper1= dirLib_upper1.resolve()

fullpathModelFile= str(dirLib_upper1)+'\\'+fullpathModel.replace('.','\\') +'.mo'
fullpathModelFile= fullpathModelFile.replace('\\', '/')


print("########## check user settings and related info. ##########")
print('-. This OS is: ' + typeOS)
print('-. Home directory of current environment is: ' + dirHome)
print("-. Temporary Working directory is: "+str(dirWork))
print('')
print('-. Directory of This notebook: '+ pathNotebook)
print('-. Directory of library which contains this notebook is: ' + str(dirLib))
print("-. Full path of Model to be ran is: " + fullpathModel)
print("-. Full path of Model file to be ran is: " + fullpathModelFile)

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

------------------------------

########## check user settings and related info. ##########
-. This OS is: Linux or Mac (posix)
-. Home directory of current environment is: /home/ubuntu
-. Temporary Working directory is: /home/ubuntu/OMPythonWork

-. Directory of This notebook: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib
-. Directory of library which contains this notebook is: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib
-. Full path of Model to be ran is: PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01
-. Full path of Model file to be ran is: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/PropulsionSystem/Examples/Engines/Transient/LycomO360_ex01.mo

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib
------------------------------


In [5]:
#**************************************************
# ----- setup -----
#**************************************************
#****************************************
#    User must NOT edit code below 
#****************************************
omc= OMCSessionZMQ()

#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

#-- move OMShell to working directory
strcmd= 'cd("' + dirWork + '")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

#--------------------
#    set build option for old frontend
#    this option setting is neccesarry so that model written with older than Modelica 4.0.0 is ran by OpenModelica newer than 1.17.0
#--------------------
print(''); print('...front end setting, wait...')
strcmd='disableNewInstantiation()'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

# --- get error message from OMShell ---
print('')
errMsg= omc.sendExpression('getErrorString()')
print('OMShell error message: ')
print(str(errMsg))
print('')

#----------------------------------------
#    settings of command line options
#----------------------------------------
print(''); print('...front end setting, wait...')
strcmd='setCommandLineOptions("--matchingAlgorithm=PFPlusExt --indexReductionMethod=dummyDerivatives -d=initialization,evaluateAllParameters,NLSanalyticJacobian")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

# --- get error message from OMShell ---
print('')
errMsg= omc.sendExpression('getErrorString()')
print('OMShell error message: ')
print(str(errMsg))
print('')

#----------------------------------------
#-- load Modelica Standard Library
#----------------------------------------
msgOMShell=omc.sendExpression("loadModel(Modelica)")
print("OMShell: "+ str(msgOMShell)); print('')

print('...loading custom libraries, wait...')
for i in range(len(libs2load)):
    #-- load PropulsionSystem library
    print('i='+str(i))
    strcmd='loadFile("' + str(dirLib) +'/'+ libs2load[i] +'/package.mo' + '")'
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))
##### end for #####
print('...load of custom libraries is completed')

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

currend dir: /home/ubuntu/OMPythonWork
------------------------------

send: cd("/home/ubuntu/OMPythonWork")
OMShell: /home/ubuntu/OMPythonWork

...front end setting, wait...
send: disableNewInstantiation()
OMShell: True

OMShell error message: 



...front end setting, wait...
send: setCommandLineOptions("--matchingAlgorithm=PFPlusExt --indexReductionMethod=dummyDerivatives -d=initialization,evaluateAllParameters,NLSanalyticJacobian")
OMShell: True

OMShell error message: 


OMShell: True

...loading custom libraries, wait...
i=0
send: loadFile("/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/PropulsionSystem/package.mo")
OMShell: True
i=1
send: loadFile("/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/FluidSystemComponents/package.mo")
OMShell: True
i=2
send: loadFile("/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/WalkingInWorldOfThermoFluid/package.mo")
OMShell: True
i=3
send: loadFile("/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/C

<h1>Building Model</h1>

In [6]:
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

#-- move OMShell to working directory
print(''); print('...moving OMshell to temporary working directry')
strcmd= 'cd("' + dirWork + '")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

#----------------------------------------
#    build the model
#----------------------------------------
print(''); print('...building the model, wait...')
strcmd= 'buildModel(' + fullpathModel + ', stopTime='+str(stopTime) + ', outputFormat="csv"'  + ')'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))
print('...model build finished.')
# --- get error message from OMShell ---
print('')
errMsg= omc.sendExpression('getErrorString()')
print('OMShell error message: ')
print(str(errMsg))
print('')


#-- save original parameter setting file
print(''); print('...Saving original parameter setting file...')
strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init.xml') +' '+ (fullpathModel+'_init_org.xml') +'")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

currend dir: /home/ubuntu/OMPythonWork
------------------------------


...moving OMshell to temporary working directry
send: cd("/home/ubuntu/OMPythonWork")
OMShell: /home/ubuntu/OMPythonWork

...building the model, wait...
send: buildModel(PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01, stopTime=90.0, outputFormat="csv")
OMShell: ('/home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01', 'PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml')
...model build finished.

OMShell error message: 
Notification: PropulsionSystem requested package Modelica of version 3.2.2. Modelica 3.2.3 is used instead which states that it is fully compatible without conversion script needed.
[/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/FluidSystemComponents/CommonAnyFluid/Components/OrificeVariableAreaCd00.mo:162:3-176:9:writable] Warning: In relation m_flow_max == port_2.m_flow,  ==  on Real numbers is only allowed inside function

<h1>Print source code of model built</h1>

In [7]:
#****************************************
#    User must NOT edit code below 
#****************************************
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

strcmd= 'list('+fullpathModel+')'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print('source code of model build:')
print('')
print(str(msgOMShell)); 

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

currend dir: /home/ubuntu/OMPythonWork
------------------------------

send: list(PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01)
source code of model build:

model LycomO360_ex01
  extends Modelica.Icons.Example;
  //-----
  package engineFluid = Modelica.Media.Air.DryAirNasa;
  //package engineFluid = PropulsionSystem.Media.EngineBreathingAir.DryAirMethaneMixture00;
  //redeclare package Medium = engineFluid
  //-----
  inner Modelica.Fluid.System system(allowFlowReversal = false) annotation(
    Placement(visible = true, transformation(origin = {-90, 180}, extent = {{-10, -10}, {10, 10}}, rotation = 0)));
  Modelica.Blocks.Sources.Ramp ramp_fracFuel(duration = 10, height = 0.0, offset = 0.056, startTime = 70) annotation(
    Placement(visible = true, transformation(origin = {-180, -100}, extent = {{-10, -10}, {10, 10}}, rotation = 0)));
  Modelica.Blocks.Sources.Ramp ramp_MN(duration = 10, height = 0, offset = 0, startTime = 30) annotation(
    Placement(visible = true, 

<h1>Run simulation, default setting</h1>

In [25]:
iRun=0
startTime=0
stopTime=90
stepSize=0.1
#----------
numberOfIntervals=int((stopTime-startTime)/stepSize)
print('numberOfIntervals of simulation='+str(numberOfIntervals))
print('')

#****************************************
#    User must NOT edit code below 
#****************************************
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

#-- reset _init.xml
print('...calling original parameter setting file...')
strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init_org.xml') +' '+ (fullpathModel+'_init.xml') +'")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell)); print('')

#-- run simulation with default parameter setting
print("...running simulation, wait...")
if(os.name=='nt'):
    strcmd= 'system("' + fullpathModel + '.exe'  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res.csv' +'")'
elif(os.name=='posix'):
    strcmd= 'system("' + './'+fullpathModel  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res.csv'  +'")'
##### end if #####
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

# --- get error message from OMShell ---
print('')
errMsg= omc.sendExpression('getErrorString()')
print('OMShell error message: ')
print(str(errMsg))
print('')

print('...simulation run finished.')

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

numberOfIntervals of simulation=900

currend dir: /home/ubuntu/OMPythonWork
------------------------------

...calling original parameter setting file...
send: system("cp PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init_org.xml PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml")
OMShell: 0

...running simulation, wait...
send: system("./PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01 -override=startTime=0,stopTime=90,stepSize=0.1 -r=PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res.csv")
OMShell: 0

OMShell error message: 


...simulation run finished.

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib
------------------------------


<h1>Run simulation</h1>
<h2>parameter sweep, 1 parameter </h2>

In [9]:
#****************************************
#    User Setting section 
#****************************************
iRunBegin=2
iRunEnd=5
param2change="inertia1.J"
valBegin=1.0
deltaVal=1.0

startTime=0
stopTime=90
stepSize=0.2

#----------
numberOfIntervals=int((stopTime-startTime)/stepSize)
print('numberOfIntervals of simulation='+str(numberOfIntervals))
print('')

#****************************************
#    User must NOT edit code below 
#****************************************
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

for i in range(iRunBegin, iRunEnd+1):
    iRun=i
    value2set=valBegin+(i-1)*deltaVal
    filenameXmlOrg=fullpathModel+'_init_org.xml'
    filenameXmlNew=fullpathModel+'_init_'+str(iRun)+'.xml'
    
    print(''); print('-----')
    print('iRun= '+ str(iRun))
    
    #-- overwrite parameter setting xml file
    print('...Overwriting parameter setting file...')
    strcmd= 'setInitXmlStartValue("' + filenameXmlOrg+'", '+ '"'+param2change+'", ' + '"'+str(value2set)+'", '+ '"'+fullpathModel+'_init.xml"' +')'
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))

    #-- save parameter setting file after overwriting
    print(''); print('...Saving parameter setting file...')
    strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init.xml') +' '+ (filenameXmlNew) +'")'
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))

    #-- run simulation with default parameter setting
    print(''); print("...running simulation, wait...")
    if(os.name=='nt'):
        strcmd= 'system("' + fullpathModel + '.exe'  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res_'+ str(iRun) +'.csv' +'")'
    elif(os.name=='posix'):
        strcmd= 'system("' + './'+fullpathModel  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res_'+ str(iRun) +'.csv'  +'")'
    ##### end if #####
    
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))
    print('...simulation run finished.')
##### end for #####

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

numberOfIntervals of simulation=450

currend dir: /home/ubuntu/OMPythonWork
------------------------------


-----
iRun= 2
...Overwriting parameter setting file...
send: setInitXmlStartValue("PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init_org.xml", "inertia1.J", "2.0", "PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml")
OMShell: True

...Saving parameter setting file...
send: system("cp PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init_2.xml")
OMShell: 0

...running simulation, wait...
send: system("./PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01 -override=startTime=0,stopTime=90,stepSize=0.2 -r=PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_res_2.csv")
OMShell: 0
...simulation run finished.

-----
iRun= 3
...Overwriting parameter setting file...
send: setInitXmlStartValue("PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init_org.xml"

<h1>Run simulation</h1>
<h2>Change multiple parameters </h2>

In [10]:
#****************************************
#    User Setting section 
#****************************************
iRun=1
param2change=[
    'ramp_throttle.height',
    'ramp_throttle.offset',
]
value2set=[
    0.4,
    0.6,
]

startTime=0
stopTime=90
stepSize=0.2

#----------
numberOfIntervals=int((stopTime-startTime)/stepSize)
print('numberOfIntervals of simulation='+str(numberOfIntervals))
print('')


#****************************************
#    User must NOT edit code below 
#****************************************
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

filenameXmlNew=fullpathModel+'_init_'+str(iRun)+'.xml'
#-- reset _init.xml
print('iRun='+str(iRun))
print('...Saving parameter setting file...')
strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init_org.xml') +' '+ (fullpathModel+'_init.xml') +'")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell)); print('')

print('...editing parameter setting file, wait...')
for i in range(len(param2change)):
    #----------
    #-- overwrite parameter setting xml file
    #- index 0
    print('i='+str(i))
    strcmd= 'setInitXmlStartValue("' +(fullpathModel+'_init.xml')+'", '+ '"'+param2change[i]+'", ' + '"'+str(value2set[i])+'", '+ '"'+(fullpathModel+'_init.xml')+'")'
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))
##### end for #####
print('...edit of parameter setting file is completed.')

#----------
#-- save parameter setting file after overwriting
print(''); print('...Saving parameter setting file...')
strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init.xml') +' '+ (filenameXmlNew) +'")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

#-- run simulation
print(''); print("...running simulation, wait...")
if(os.name=='nt'):
    strcmd= 'system("' + fullpathModel + '.exe'  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res_'+ str(iRun) +'.csv' +'")'
elif(os.name=='posix'):
    strcmd= 'system("' + './'+fullpathModel  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res_'+ str(iRun) +'.csv'  +'")'
##### end if #####

print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))
print('...simulation run finished.')

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

numberOfIntervals of simulation=450

currend dir: /home/ubuntu/OMPythonWork
------------------------------

iRun=1
...Saving parameter setting file...
send: system("cp PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init_org.xml PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml")
OMShell: 0

...editing parameter setting file, wait...
i=0
send: setInitXmlStartValue("PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml", "ramp_throttle.height", "0.4", "PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml")
OMShell: True
i=1
send: setInitXmlStartValue("PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml", "ramp_throttle.offset", "0.6", "PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml")
OMShell: True
...edit of parameter setting file is completed.

...Saving parameter setting file...
send: system("cp PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_init.xml PropulsionSystem.Examples.Eng

<h1>---------------------------------------------------------------------------------------------------------</h1>

<h1>Post Process --- Read csv data into table object ---</h1>

<h2>Read csv, case of default parameter setting</h2>

In [11]:
i_df=0

#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

nmcsv= fullpathModel + '_res.csv'
print("data read: "+ os.getcwd()+"/"+nmcsv)
df.insert(i_df, pd.read_csv(nmcsv))

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())

df[i_df]

currend dir: /home/ubuntu/OMPythonWork
------------------------------

data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_res.csv

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib


,time,damper1.phi_rel,damper1.w_rel,der(damper1.phi_rel),der(damper1.w_rel),Flt2Fluid.AltMN2pTh.TambStd,Flt2Fluid.AltMN2pTh.fluidAmb.MM,Flt2Fluid.AltMN2pTh.fluidAmb.R,Flt2Fluid.AltMN2pTh.fluidAmb.T_degC,Flt2Fluid.AltMN2pTh.fluidAmb.X[1],...,throttle.port_1.h_outflow,throttle.port_1.m_flow,throttle.port_1.p,throttle.port_2.m_flow,throttle.port_2.p,throttle.u_Cd,throttle.u_kArea,to_rpm_crankshaft.u,torque1.flange.phi,torque1.tau
0,0.0,0.000000,-209.439510,-209.439510,-9.138144e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.099558,101323.7,-0.099558,83520.541741,0.6,0.8,209.439510,-0.000000,0
1,0.1,-25.101554,-288.822657,-288.822657,-6.872731e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.122356,101323.7,-0.122356,74433.550807,0.6,0.8,288.822657,25.101554,0
2,0.2,-57.131534,-349.127004,-349.127004,-5.278158e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.135656,101323.7,-0.135656,68270.006634,0.6,0.8,349.127004,57.131534,0
3,0.3,-94.476104,-395.846368,-395.846368,-4.124697e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.144169,101323.7,-0.144169,63991.200650,0.6,0.8,395.846368,94.476104,0
4,0.4,-135.969591,-432.594523,-432.594523,-3.265021e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.149986,101323.7,-0.149986,60917.844366,0.6,0.8,432.594523,135.969591,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,89.7,-55701.803961,-640.483960,-640.483960,-5.395577e-10,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.200649,101323.7,-0.200649,55043.239108,0.6,1.0,640.483960,55701.803961,0
902,89.8,-55765.852357,-640.483960,-640.483960,-5.320544e-10,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.200649,101323.7,-0.200649,55043.239108,0.6,1.0,640.483960,55765.852357,0
903,89.9,-55829.900753,-640.483960,-640.483960,-5.302354e-10,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.200649,101323.7,-0.200649,55043.239108,0.6,1.0,640.483960,55829.900753,0
904,90.0,-55893.949149,-640.483960,-640.483960,-5.313723e-10,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.200649,101323.7,-0.200649,55043.239108,0.6,1.0,640.483960,55893.949149,0


<h1> list of columns on csv </h1>

In [12]:
i_df=0

print('list of variables on csv')
df[i_df].columns.tolist()


list of variables on csv


['time',
 'damper1.phi_rel',
 'damper1.w_rel',
 'der(damper1.phi_rel)',
 'der(damper1.w_rel)',
 'Flt2Fluid.AltMN2pTh.TambStd',
 'Flt2Fluid.AltMN2pTh.fluidAmb.MM',
 'Flt2Fluid.AltMN2pTh.fluidAmb.R',
 'Flt2Fluid.AltMN2pTh.fluidAmb.T_degC',
 'Flt2Fluid.AltMN2pTh.fluidAmb.X[1]',
 'Flt2Fluid.AltMN2pTh.fluidAmb.d',
 'Flt2Fluid.AltMN2pTh.fluidAmb.p_bar',
 'Flt2Fluid.AltMN2pTh.fluidAmb.u',
 'Flt2Fluid.AltMN2pTh.fluidAmbStd.MM',
 'Flt2Fluid.AltMN2pTh.fluidAmbStd.R',
 'Flt2Fluid.AltMN2pTh.fluidAmbStd.T_degC',
 'Flt2Fluid.AltMN2pTh.fluidAmbStd.X[1]',
 'Flt2Fluid.AltMN2pTh.fluidAmbStd.d',
 'Flt2Fluid.AltMN2pTh.fluidAmbStd.h',
 'Flt2Fluid.AltMN2pTh.fluidAmbStd.p_bar',
 'Flt2Fluid.AltMN2pTh.fluidAmbStd.u',
 'Flt2Fluid.AltMN2pTh.fluidTot.MM',
 'Flt2Fluid.AltMN2pTh.fluidTot.R',
 'Flt2Fluid.AltMN2pTh.fluidTot.T_degC',
 'Flt2Fluid.AltMN2pTh.fluidTot.X[1]',
 'Flt2Fluid.AltMN2pTh.fluidTot.d',
 'Flt2Fluid.AltMN2pTh.fluidTot.p_bar',
 'Flt2Fluid.AltMN2pTh.fluidTot.u',
 'Flt2Fluid.AltMN2pTh.pAmbStd',
 'Flt2Fl

<h1>Read csv, array, specify start and end</h1>

In [13]:
i_df_begin=2
i_df_end=5


#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

for i in range(i_df_begin, i_df_end+1):
    i_df=i
    print('i_df='+str(i_df))
    nmcsv= fullpathModel + '_res_'+str(i_df)+'.csv'
    print("data read: "+ os.getcwd()+"/"+nmcsv)
    df.insert(i_df, pd.read_csv(nmcsv))
##### end for #####
    
#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())


currend dir: /home/ubuntu/OMPythonWork
------------------------------

i_df=2
data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_res_2.csv
i_df=3
data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_res_3.csv
i_df=4
data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_res_4.csv
i_df=5
data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_res_5.csv

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib


<h1>Read csv, selected case#</h1>

In [14]:
i_df=1

#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

nmcsv= fullpathModel + '_res_'+str(i_df)+'.csv'
print("data read: "+ os.getcwd()+"/"+nmcsv)
df.insert(i_df, pd.read_csv(nmcsv))

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())

df[i_df]

currend dir: /home/ubuntu/OMPythonWork
------------------------------

data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360_ex01_res_1.csv

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib


,time,damper1.phi_rel,damper1.w_rel,der(damper1.phi_rel),der(damper1.w_rel),Flt2Fluid.AltMN2pTh.TambStd,Flt2Fluid.AltMN2pTh.fluidAmb.MM,Flt2Fluid.AltMN2pTh.fluidAmb.R,Flt2Fluid.AltMN2pTh.fluidAmb.T_degC,Flt2Fluid.AltMN2pTh.fluidAmb.X[1],...,throttle.port_1.h_outflow,throttle.port_1.m_flow,throttle.port_1.p,throttle.port_2.m_flow,throttle.port_2.p,throttle.u_Cd,throttle.u_kArea,to_rpm_crankshaft.u,torque1.flange.phi,torque1.tau
0,0.0,0.000000,-209.439510,-209.439510,-9.138144e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.099558,101323.7,-0.099558,83520.541741,0.6,0.8,209.439510,-0.000000,0
1,0.2,-57.131534,-349.127004,-349.127004,-5.278158e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.135656,101323.7,-0.135656,68270.006634,0.6,0.8,349.127004,57.131534,0
2,0.4,-135.969591,-432.594523,-432.594523,-3.265021e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.149986,101323.7,-0.149986,60917.844366,0.6,0.8,432.594523,135.969591,0
3,0.6,-228.141666,-485.262490,-485.262490,-2.099615e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.157221,101323.7,-0.157221,56925.699289,0.6,0.8,485.262490,228.141666,0
4,0.8,-328.857815,-519.508948,-519.508948,-1.379983e+02,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.161343,101323.7,-0.161343,54567.175023,0.6,0.8,519.508948,328.857815,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,89.4,-55509.658773,-640.483960,-640.483960,-5.436505e-10,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.200649,101323.7,-0.200649,55043.239108,0.6,1.0,640.483960,55509.658773,0
452,89.6,-55637.755565,-640.483960,-640.483960,-5.412630e-10,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.200649,101323.7,-0.200649,55043.239108,0.6,1.0,640.483960,55637.755565,0
453,89.8,-55765.852357,-640.483960,-640.483960,-5.320544e-10,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.200649,101323.7,-0.200649,55043.239108,0.6,1.0,640.483960,55765.852357,0
454,90.0,-55893.949149,-640.483960,-640.483960,-5.306902e-10,288.16,0.028965,287.051225,15.01,1,...,288574.819851,0.200649,101323.7,-0.200649,55043.239108,0.6,1.0,640.483960,55893.949149,0


<h1>===========================================================================</h1>

<h1>Run simulation model #2</h1>

<h1>User settings: Library and Model information</h1>
<p>Edit the strings below </p>

In [15]:
dirWorkRelToHome='/OMPythonWork'

#dirModelRelToLibDir='/Examples/ThermodynamicCycles'
libs2load=["PropulsionSystem", "FluidSystemComponents", "WalkingInWorldOfThermoFluid", "AircraftDynamics", "SystemModels", "InteractiveSimulation"]
fullpathModel='PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01'
nmModel='LycomO360turboCharged_ex01'
stopTime=90.0
outputFormat='csv'
print('------------------------------')

------------------------------


<h1>Setup </h1>

In [17]:
#**************************************************
# ----- setup -----
# for 2nd model and lator
#**************************************************

fullpathModelFile= str(dirLib_upper1)+'\\'+fullpathModel.replace('.','\\') +'.mo'
fullpathModelFile= fullpathModelFile.replace('\\', '/')

print("########## check user settings and related info. ##########")
print('-. This OS is: ' + typeOS)
print('-. Home directory of current environment is: ' + dirHome)
print("-. Temporary Working directory is: "+str(dirWork))
print('')
print('-. Directory of This notebook: '+ pathNotebook)
print('-. Directory of library which contains this notebook is: ' + str(dirLib))
print("-. Full path of Model to be ran is: " + fullpathModel)
print("-. Full path of Model file to be ran is: " + fullpathModelFile)

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

########## check user settings and related info. ##########
-. This OS is: Linux or Mac (posix)
-. Home directory of current environment is: /home/ubuntu
-. Temporary Working directory is: /home/ubuntu/OMPythonWork

-. Directory of This notebook: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib
-. Directory of library which contains this notebook is: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib
-. Full path of Model to be ran is: PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01
-. Full path of Model file to be ran is: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/PropulsionSystem/Examples/Engines/Transient/LycomO360turboCharged_ex01.mo

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib
------------------------------


In [18]:
#**************************************************
# ----- setup -----
#**************************************************
#****************************************
#    User must NOT edit code below 
#****************************************
omc= OMCSessionZMQ()

#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

#-- move OMShell to working directory
strcmd= 'cd("' + dirWork + '")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

#--------------------
#    set build option for old frontend
#    this option setting is neccesarry so that model written with older than Modelica 4.0.0 is ran by OpenModelica newer than 1.17.0
#--------------------
print(''); print('...front end setting, wait...')
strcmd='disableNewInstantiation()'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

# --- get error message from OMShell ---
print('')
errMsg= omc.sendExpression('getErrorString()')
print('OMShell error message: ')
print(str(errMsg))
print('')

#----------------------------------------
#    settings of command line options
#----------------------------------------
print(''); print('...front end setting, wait...')
strcmd='setCommandLineOptions("--matchingAlgorithm=PFPlusExt --indexReductionMethod=dummyDerivatives -d=initialization,evaluateAllParameters,NLSanalyticJacobian")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

# --- get error message from OMShell ---
print('')
errMsg= omc.sendExpression('getErrorString()')
print('OMShell error message: ')
print(str(errMsg))
print('')

#----------------------------------------
#-- load Modelica Standard Library
#----------------------------------------
msgOMShell=omc.sendExpression("loadModel(Modelica)")
print("OMShell: "+ str(msgOMShell)); print('')

print('...loading custom libraries, wait...')
for i in range(len(libs2load)):
    #-- load PropulsionSystem library
    print('i='+str(i))
    strcmd='loadFile("' + str(dirLib) +'/'+ libs2load[i] +'/package.mo' + '")'
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))
##### end for #####
print('...load of custom libraries is completed')

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

currend dir: /home/ubuntu/OMPythonWork
------------------------------

send: cd("/home/ubuntu/OMPythonWork")
OMShell: /home/ubuntu/OMPythonWork

...front end setting, wait...
send: disableNewInstantiation()
OMShell: True

OMShell error message: 



...front end setting, wait...
send: setCommandLineOptions("--matchingAlgorithm=PFPlusExt --indexReductionMethod=dummyDerivatives -d=initialization,evaluateAllParameters,NLSanalyticJacobian")
OMShell: True

OMShell error message: 


OMShell: True

...loading custom libraries, wait...
i=0
send: loadFile("/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/PropulsionSystem/package.mo")
OMShell: True
i=1
send: loadFile("/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/FluidSystemComponents/package.mo")
OMShell: True
i=2
send: loadFile("/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/WalkingInWorldOfThermoFluid/package.mo")
OMShell: True
i=3
send: loadFile("/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/C

<h1>Print source code of model built</h1>

In [19]:
#****************************************
#    User must NOT edit code below 
#****************************************
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

strcmd= 'list('+fullpathModel+')'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print('source code of model build:')
print('')
print(str(msgOMShell)); 

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

currend dir: /home/ubuntu/OMPythonWork
------------------------------

send: list(PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01)
source code of model build:

model LycomO360turboCharged_ex01
  extends Modelica.Icons.Example;
  //-----
  package engineFluid = Modelica.Media.Air.DryAirNasa;
  //package engineFluid = PropulsionSystem.Media.EngineBreathingAir.DryAirMethaneMixture00;
  //redeclare package Medium = engineFluid
  //-----
  inner Modelica.Fluid.System system(allowFlowReversal = false) annotation(
    Placement(visible = true, transformation(origin = {-90, 180}, extent = {{-10, -10}, {10, 10}}, rotation = 0)));
  Modelica.Blocks.Sources.Ramp ramp_fracFuel(duration = 10, height = 0.0, offset = 0.056, startTime = 70) annotation(
    Placement(visible = true, transformation(origin = {-180, -100}, extent = {{-10, -10}, {10, 10}}, rotation = 0)));
  Modelica.Blocks.Sources.Ramp ramp_MN(duration = 10, height = 0, offset = 0, startTime = 30) annotation(
    Pl

<h1>Building Model</h1>

In [20]:
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

#-- move OMShell to working directory
print(''); print('...moving OMshell to temporary working directry')
strcmd= 'cd("' + dirWork + '")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

#----------------------------------------
#    build the model
#----------------------------------------
print(''); print('...building the model, wait...')
strcmd= 'buildModel(' + fullpathModel + ', stopTime='+str(stopTime) + ', outputFormat="csv"'  + ')'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))
print('...model build finished.')
# --- get error message from OMShell ---
print('')
errMsg= omc.sendExpression('getErrorString()')
print('OMShell error message: ')
print(str(errMsg))
print('')


#-- save original parameter setting file
print(''); print('...Saving original parameter setting file...')
strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init.xml') +' '+ (fullpathModel+'_init_org.xml') +'")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

currend dir: /home/ubuntu/OMPythonWork
------------------------------


...moving OMshell to temporary working directry
send: cd("/home/ubuntu/OMPythonWork")
OMShell: /home/ubuntu/OMPythonWork

...building the model, wait...
send: buildModel(PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01, stopTime=90.0, outputFormat="csv")
OMShell: ('/home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01', 'PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml')
...model build finished.

OMShell error message: 
Notification: PropulsionSystem requested package Modelica of version 3.2.2. Modelica 3.2.3 is used instead which states that it is fully compatible without conversion script needed.
[/home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/FluidSystemComponents/CommonAnyFluid/Components/OrificeVariableAreaCd00.mo:162:3-176:9:writable] Warning: In relation m_flow_max == port_2.m_flow,  ==  on Real num

<h1>Run simulation, default setting</h1>

In [28]:
iRun=100
startTime=0
stopTime=100
stepSize=0.1
#----------
numberOfIntervals=int((stopTime-startTime)/stepSize)
print('numberOfIntervals of simulation='+str(numberOfIntervals))
print('')

#****************************************
#    User must NOT edit code below 
#****************************************
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

#-- reset _init.xml
print('...calling original parameter setting file...')
strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init_org.xml') +' '+ (fullpathModel+'_init.xml') +'")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell)); print('')

#-- run simulation with default parameter setting
print("...running simulation, wait...")
if(os.name=='nt'):
    strcmd= 'system("' + fullpathModel + '.exe'  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res.csv' +'")'
elif(os.name=='posix'):
    strcmd= 'system("' + './'+fullpathModel  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res.csv'  +'")'
##### end if #####
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

# --- get error message from OMShell ---
print('')
errMsg= omc.sendExpression('getErrorString()')
print('OMShell error message: ')
print(str(errMsg))
print('')

print('...simulation run finished.')

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

numberOfIntervals of simulation=1000

currend dir: /home/ubuntu/OMPythonWork
------------------------------

...calling original parameter setting file...
send: system("cp PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init_org.xml PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml")
OMShell: 0

...running simulation, wait...
send: system("./PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01 -override=startTime=0,stopTime=100,stepSize=0.1 -r=PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res.csv")
OMShell: 0

OMShell error message: 


...simulation run finished.

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib
------------------------------


<h1>Run simulation</h1>
<h2>parameter sweep, 1 parameter </h2>

In [29]:
#****************************************
#    User Setting section 
#****************************************
iRunBegin=102
iRunEnd=105
param2change="inertia1.J"
valBegin=1.0
deltaVal=1.0

startTime=0
stopTime=100
stepSize=0.2

#----------
numberOfIntervals=int((stopTime-startTime)/stepSize)
print('numberOfIntervals of simulation='+str(numberOfIntervals))
print('')

#****************************************
#    User must NOT edit code below 
#****************************************
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

for i in range(iRunBegin, iRunEnd+1):
    iRun=i
    value2set=valBegin+(i-1)*deltaVal
    filenameXmlOrg=fullpathModel+'_init_org.xml'
    filenameXmlNew=fullpathModel+'_init_'+str(iRun)+'.xml'
    
    print(''); print('-----')
    print('iRun= '+ str(iRun))
    
    #-- overwrite parameter setting xml file
    print('...Overwriting parameter setting file...')
    strcmd= 'setInitXmlStartValue("' + filenameXmlOrg+'", '+ '"'+param2change+'", ' + '"'+str(value2set)+'", '+ '"'+fullpathModel+'_init.xml"' +')'
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))

    #-- save parameter setting file after overwriting
    print(''); print('...Saving parameter setting file...')
    strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init.xml') +' '+ (filenameXmlNew) +'")'
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))

    #-- run simulation with default parameter setting
    print(''); print("...running simulation, wait...")
    if(os.name=='nt'):
        strcmd= 'system("' + fullpathModel + '.exe'  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res_'+ str(iRun) +'.csv' +'")'
    elif(os.name=='posix'):
        strcmd= 'system("' + './'+fullpathModel  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res_'+ str(iRun) +'.csv'  +'")'
    ##### end if #####
    
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))
    print('...simulation run finished.')
##### end for #####

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

numberOfIntervals of simulation=500

currend dir: /home/ubuntu/OMPythonWork
------------------------------


-----
iRun= 102
...Overwriting parameter setting file...
send: setInitXmlStartValue("PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init_org.xml", "inertia1.J", "102.0", "PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml")
OMShell: True

...Saving parameter setting file...
send: system("cp PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init_102.xml")
OMShell: 0

...running simulation, wait...
send: system("./PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01 -override=startTime=0,stopTime=100,stepSize=0.2 -r=PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res_102.csv")
OMShell: 0
...simulation run finished.

-----
iRun= 103
...Overwriting parameter setting file...
send: setInitXmlS

<h1>Run simulation</h1>
<h2>Change multiple parameters </h2>

In [30]:
#****************************************
#    User Setting section 
#****************************************
iRun=101
param2change=[
    'ramp_throttle.height',
    'ramp_throttle.offset',
]
value2set=[
    0.4,
    0.6,
]

startTime=0
stopTime=100
stepSize=0.2

#----------
numberOfIntervals=int((stopTime-startTime)/stepSize)
print('numberOfIntervals of simulation='+str(numberOfIntervals))
print('')


#****************************************
#    User must NOT edit code below 
#****************************************
#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

filenameXmlNew=fullpathModel+'_init_'+str(iRun)+'.xml'
#-- reset _init.xml
print('iRun='+str(iRun))
print('...Saving parameter setting file...')
strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init_org.xml') +' '+ (fullpathModel+'_init.xml') +'")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell)); print('')

print('...editing parameter setting file, wait...')
for i in range(len(param2change)):
    #----------
    #-- overwrite parameter setting xml file
    #- index 0
    print('i='+str(i))
    strcmd= 'setInitXmlStartValue("' +(fullpathModel+'_init.xml')+'", '+ '"'+param2change[i]+'", ' + '"'+str(value2set[i])+'", '+ '"'+(fullpathModel+'_init.xml')+'")'
    print('send: '+strcmd)
    msgOMShell=omc.sendExpression(strcmd)
    print("OMShell: "+ str(msgOMShell))
##### end for #####
print('...edit of parameter setting file is completed.')

#----------
#-- save parameter setting file after overwriting
print(''); print('...Saving parameter setting file...')
strcmd= 'system("'+ 'cp '+ (fullpathModel+'_init.xml') +' '+ (filenameXmlNew) +'")'
print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))

#-- run simulation
print(''); print("...running simulation, wait...")
if(os.name=='nt'):
    strcmd= 'system("' + fullpathModel + '.exe'  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res_'+ str(iRun) +'.csv' +'")'
elif(os.name=='posix'):
    strcmd= 'system("' + './'+fullpathModel  + ' -override=startTime='+str(startTime)+',stopTime='+str(stopTime)+',stepSize='+str(stepSize)  + ' -r=' + fullpathModel + '_res_'+ str(iRun) +'.csv'  +'")'
##### end if #####

print('send: '+strcmd)
msgOMShell=omc.sendExpression(strcmd)
print("OMShell: "+ str(msgOMShell))
print('...simulation run finished.')

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')

numberOfIntervals of simulation=500

currend dir: /home/ubuntu/OMPythonWork
------------------------------

iRun=101
...Saving parameter setting file...
send: system("cp PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init_org.xml PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml")
OMShell: 0

...editing parameter setting file, wait...
i=0
send: setInitXmlStartValue("PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml", "ramp_throttle.height", "0.4", "PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml")
OMShell: True
i=1
send: setInitXmlStartValue("PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml", "ramp_throttle.offset", "0.6", "PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_init.xml")
OMShell: True
...edit of parameter setting file is completed.

...Saving parameter setting file...
send: system("cp PropulsionSystem.Exampl

<h1>---------------------------------------------------------------------------------------------------------</h1>

<h1>Post Process --- Read csv data into table object ---</h1>

<h2>Read csv, case of default parameter setting</h2>

In [31]:
i_df=100

#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

nmcsv= fullpathModel + '_res.csv'
print("data read: "+ os.getcwd()+"/"+nmcsv)
df.insert(i_df, pd.read_csv(nmcsv))

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())

df[i_df]

currend dir: /home/ubuntu/OMPythonWork
------------------------------

data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res.csv

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib


,time,Trb.phi,damper1.phi_rel,damper1.w_rel,inertia2.w,der(Trb.phi),der(damper1.phi_rel),der(damper1.w_rel),der(inertia2.w),der(Cmp.phi),...,throttle.port_1.m_flow,throttle.port_1.p,throttle.port_2.m_flow,throttle.port_2.p,throttle.u_Cd,throttle.u_kArea,to_rpm_crankshaft.u,to_rpm_turboCharger.u,torque1.flange.phi,torque1.tau
0,0.0,1.000000,0.000000,-209.439510,3141.592654,3141.592654,-209.439510,-3377.579981,-121.146853,3141.592654,...,0.309445,506618.500000,-0.309445,447342.292159,0.6,0.8,209.439510,3141.592654,-0.000000,0
1,0.1,314.810194,-33.081430,-420.651931,3136.593612,3136.593612,-420.651931,-1380.921525,0.214878,3136.593612,...,0.346947,327641.600725,-0.346947,224199.043705,0.6,0.8,420.651931,3136.593612,33.081430,0
2,0.1,314.810194,-33.081430,-420.651931,3136.593612,3136.593612,-420.651931,-1380.921524,0.214878,3136.593612,...,0.346947,327641.600649,-0.346947,224199.043611,0.6,0.8,420.651931,3136.593612,33.081430,0
3,0.1,314.810194,-33.081430,-420.651931,3136.593612,3136.593612,-420.651931,-1380.921524,0.214878,3136.593612,...,0.346947,327641.600649,-0.346947,224199.043611,0.6,0.8,420.651931,3136.593612,33.081430,0
4,0.2,628.565568,-80.931857,-527.396835,3139.289429,3139.289429,-527.396835,-832.448514,47.472366,3139.289429,...,0.348331,286949.412000,-0.348331,172535.063971,0.6,0.8,527.396835,3139.289429,80.931857,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,99.7,435208.147963,-92569.729472,-955.078837,4581.670774,4581.670774,-955.078837,-0.000009,0.000079,4581.670774,...,0.589020,376764.591640,-0.589020,189345.601264,0.6,1.0,955.078837,4581.670774,92569.729472,0
1004,99.8,435666.315062,-92665.237358,-955.078838,4581.670778,4581.670778,-955.078838,-0.000009,0.000079,4581.670778,...,0.589020,376764.592150,-0.589020,189345.601516,0.6,1.0,955.078838,4581.670778,92665.237358,0
1005,99.9,436124.482160,-92760.745244,-955.078838,4581.670781,4581.670781,-955.078838,-0.000009,0.000078,4581.670781,...,0.589020,376764.592664,-0.589020,189345.601770,0.6,1.0,955.078838,4581.670781,92760.745244,0
1006,100.0,436582.649258,-92856.253130,-955.078839,4581.670784,4581.670784,-955.078839,-0.000009,0.000077,4581.670784,...,0.589020,376764.593182,-0.589020,189345.602026,0.6,1.0,955.078839,4581.670784,92856.253130,0


<h1> list of columns on csv </h1>

In [32]:
i_df=100

print('list of variables on csv')
df[i_df].columns.tolist()


list of variables on csv


['time',
 'Trb.phi',
 'damper1.phi_rel',
 'damper1.w_rel',
 'inertia2.w',
 'der(Trb.phi)',
 'der(damper1.phi_rel)',
 'der(damper1.w_rel)',
 'der(inertia2.w)',
 'der(Cmp.phi)',
 'Cmp.CmpTbl_WcPReff_NcRline.u_NcTbl',
 'Cmp.CmpTbl_WcPReff_NcRline.y_PR',
 'Cmp.CmpTbl_WcPReff_NcRline.y_Wc',
 'Cmp.CmpTbl_WcPReff_NcRline.y_eff',
 'Cmp.Nc_1',
 'Cmp.NcqNcDes_1',
 'Cmp.Nmech',
 'Cmp.NqNdes',
 'Cmp.PR',
 'Cmp.Rline',
 'Cmp.ScalerDesCmp.CmpTbl_WcPReff_NcRline_Des.y_PR',
 'Cmp.ScalerDesCmp.CmpTbl_WcPReff_NcRline_Des.y_Wc',
 'Cmp.ScalerDesCmp.CmpTbl_WcPReff_NcRline_Des.y_eff',
 'Cmp.ScalerDesCmp.const.y',
 'Cmp.ScalerDesCmp.const1.y',
 'Cmp.ScalerDesCmp.division3.u1',
 'Cmp.ScalerDesCmp.division3.u2',
 'Cmp.ScalerDesCmp.u_NcTblDes',
 'Cmp.ScalerDesCmp.u_RlineTblDes',
 'Cmp.ScalerDesCmp.y_s_PRdes',
 'Cmp.ScalerDesCmp.y_s_WcDes',
 'Cmp.ScalerDesCmp.y_s_effDes',
 'Cmp.SclCmp.add1.u1',
 'Cmp.SclCmp.const.y',
 'Cmp.SclCmp.const1.y',
 'Cmp.SclCmp.feedback1.y',
 'Cmp.SclCmp.y_PRscld',
 'Cmp.SclCmp.y_WcScld

<h1>Read csv, array, specify start and end</h1>

In [33]:
i_df_begin=102
i_df_end=105


#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

for i in range(i_df_begin, i_df_end+1):
    i_df=i
    print('i_df='+str(i_df))
    nmcsv= fullpathModel + '_res_'+str(i_df)+'.csv'
    print("data read: "+ os.getcwd()+"/"+nmcsv)
    df.insert(i_df, pd.read_csv(nmcsv))
##### end for #####
    
#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())


currend dir: /home/ubuntu/OMPythonWork
------------------------------

i_df=102
data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res_102.csv
i_df=103
data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res_103.csv
i_df=104
data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res_104.csv
i_df=105
data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res_105.csv

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib


<h1>Read csv, selected case#</h1>

In [34]:
i_df=101

#***** move to temporary working directory *****
os.chdir(dirWork)
os.getcwd()
print("currend dir: "+os.getcwd())
print('------------------------------'); print('')

nmcsv= fullpathModel + '_res_'+str(i_df)+'.csv'
print("data read: "+ os.getcwd()+"/"+nmcsv)
df.insert(i_df, pd.read_csv(nmcsv))

#***** move back to notebook directory *****
print(''); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())

df[i_df]

currend dir: /home/ubuntu/OMPythonWork
------------------------------

data read: /home/ubuntu/OMPythonWork/PropulsionSystem.Examples.Engines.Transient.LycomO360turboCharged_ex01_res_101.csv

------------------------------
move to directory of notebook
currend dir, back to: /home/ubuntu/Dropbox/01-sharedspace/modelica-shared/CustomLib/ipynbs4OMPython/PropulsionSystemLib


,time,Trb.phi,damper1.phi_rel,damper1.w_rel,inertia2.w,der(Trb.phi),der(damper1.phi_rel),der(damper1.w_rel),der(inertia2.w),der(Cmp.phi),...,throttle.port_1.m_flow,throttle.port_1.p,throttle.port_2.m_flow,throttle.port_2.p,throttle.u_Cd,throttle.u_kArea,to_rpm_crankshaft.u,to_rpm_turboCharger.u,torque1.flange.phi,torque1.tau
0,0.0,1.000000,0.000000,-209.439510,3141.592654,3141.592654,-209.439510,-3377.579981,-121.146853,3141.592654,...,0.309445,506618.500000,-0.309445,447342.292159,0.6,0.8,209.439510,3141.592654,-0.000000,0
1,0.1,314.810194,-33.081430,-420.651931,3136.593612,3136.593612,-420.651931,-1380.921524,0.214878,3136.593612,...,0.346947,327641.600649,-0.346947,224199.043611,0.6,0.8,420.651931,3136.593612,33.081430,0
2,0.1,314.810194,-33.081430,-420.651931,3136.593612,3136.593612,-420.651931,-1380.921524,0.214878,3136.593612,...,0.346947,327641.600649,-0.346947,224199.043611,0.6,0.8,420.651931,3136.593612,33.081430,0
3,0.2,628.565568,-80.931857,-527.396835,3139.289429,3139.289429,-527.396835,-832.448514,47.472366,3139.289429,...,0.348331,286949.412000,-0.348331,172535.063971,0.6,0.8,527.396835,3139.289429,80.931857,0
4,0.4,1257.693924,-199.435187,-643.697148,3153.223991,3153.223991,-643.697148,-403.323117,85.316616,3153.223991,...,0.350812,261881.230321,-0.350812,138330.063432,0.6,0.8,643.697148,3153.223991,199.435187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,99.4,433833.646668,-92283.205813,-955.078836,4581.670765,4581.670765,-955.078836,-0.000009,0.000081,4581.670765,...,0.589020,376764.590137,-0.589020,189345.600522,0.6,1.0,955.078836,4581.670765,92283.205813,0
504,99.6,434749.980865,-92474.221585,-955.078837,4581.670771,4581.670771,-955.078837,-0.000009,0.000080,4581.670771,...,0.589020,376764.591135,-0.589020,189345.601015,0.6,1.0,955.078837,4581.670771,92474.221585,0
505,99.8,435666.315062,-92665.237358,-955.078838,4581.670778,4581.670778,-955.078838,-0.000009,0.000079,4581.670778,...,0.589020,376764.592150,-0.589020,189345.601516,0.6,1.0,955.078838,4581.670778,92665.237358,0
506,100.0,436582.649258,-92856.253130,-955.078839,4581.670784,4581.670784,-955.078839,-0.000009,0.000077,4581.670784,...,0.589020,376764.593182,-0.589020,189345.602026,0.6,1.0,955.078839,4581.670784,92856.253130,0


<h1>---------------------------------------------------------------------------------------------------------</h1>

<h1>===========================================================================</h1>

<h1>Post Process --- Visualization ---</h1>

In [ ]:
fig=[]
i_fig=-1
print('------------------------------')

<h2>Simulation results; Inputs</h2>

In [ ]:
i_df=1
varName="Orifice.zeta"
ylabel='dp coefficient [nond]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]
fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.plot(time, var)
plt.title(varName)
plt.grid()
print('------------------------------')

<h2>Simulation Results; Outputs</h2>

In [ ]:
i_df=1
varName="PistonCylinder.Nmech"
ylabel='Rotation speed [rpm]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]
fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.Nmech"
ylabel='Rotation speed [rpm]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]
fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')
plt.ylim(12100, 12300)

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.pwrOut"
ylabel='Power [W]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]
fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.pwrOut"
ylabel='Power [W]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]
fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')
plt.ylim(16000, 16500)

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.trqOut"
ylabel='Torque [N*m]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]
fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.trqOut"
ylabel='Torque [N*m]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]
fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')
plt.ylim(12.7, 12.9)

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.pwrPumping"
ylabel='power [W]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]

fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.pwrPumping"
ylabel='power [W]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]

fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')
plt.ylim(0, 12)

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.pwrPumpingqPwrCycle"
ylabel='fraction [nond]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]

fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.pwrPumpingqPwrCycle"
ylabel='fraction [nond]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]

fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')
plt.ylim(0, 0.001)

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.port_1.p"
ylabel='Pressure [kPa]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]/1000.0

fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.port_1.p"
ylabel='Pressure [kPa]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]/1000.0

fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')
plt.ylim(101, 102)

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.port_2.p"
ylabel='Pressure [kPa]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]/1000.0

fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')

plt.grid()
print('------------------------------')

In [ ]:
i_df=1
varName="PistonCylinder.port_2.p"
ylabel='Pressure [kPa]'

time= df[i_df].loc[:,'time']
var= df[i_df].loc[:,varName]/1000.0

fig.append(plt.figure(figsize=(15,8)))
plt.xlabel('time [s]')
plt.ylabel(ylabel)
plt.title(varName)
plt.plot(time, var, color='red')
plt.ylim(101, 101.5)

plt.grid()
print('------------------------------')

<h1>Extract variables at specified time</h1>

In [ ]:
i_df=1
tgtTime=45
widthTime=1

#******************************
#*** find the row of target time ***
rowTgt=0
for i in range(len(df[i_df])):
    if((tgtTime-widthTime <= df[i_df].at[i,'time']) and (df[i_df].at[i,'time'] <= tgtTime) ):
        rowTgt=i
        timeExtrd=df[i_df].at[i,'time']
    ##### end if #####
##### end for #####

print("row extracted = "+str(rowTgt))
print("time of extracted = "+str(timeExtrd))

arr_h=[]
arr_s=[]
arr_v=[]
arr_T=[]
arr_u=[]
for i in range(1,5+1):
    # ***** extract specified variables of target row *****
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_h['+str(i)+']')]
    arr_h.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_s['+str(i)+']')]
    arr_s.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_v['+str(i)+']')]
    arr_v.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_T['+str(i)+']')]
    arr_T.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_u['+str(i)+']')]
    arr_u.append(val)
##### end for #####

arr_V=[]
arr_p=[]
for i in range(1,8+1):
    # ***** extract specified variables of target row *****
    val= df[i_df].at[rowTgt, ('PistonCylinder.arr_p['+str(i)+']')]
    arr_p.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.arr_V['+str(i)+']')]
    arr_V.append(val)
##### end for #####

nPts= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_hs[1].nPts')]
s_curve_hs=[[]]*4; h_curve_hs=[[]]*4
v_curve_pv=[[]]*4; p_curve_pv=[[]]*4
s_curve_Ts=[[]]*4; T_curve_Ts=[[]]*4
s_curve_us=[[]]*4; u_curve_us=[[]]*4
V_curve_pVol=[[]]*4; p_curve_pVol=[[]]*4

arrTemp_s_hs=[]; arrTemp_h_hs=[]
arrTemp_s_Ts=[]; arrTemp_T_Ts=[]
arrTemp_s_us=[]; arrTemp_u_us=[]
arrTemp_v_pv=[]; arrTemp_p_pv=[]
arrTemp_V_pVol=[]; arrTemp_p_pVol=[]

for j in range(1, 4+1):
    for i in range(1, nPts+1):
        # ***** extract specified variables of target row *****
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_hs['+str(j)+'].arr_s['+str(i)+']')]; arrTemp_s_hs.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_hs['+str(j)+'].arr_h['+str(i)+']')]; arrTemp_h_hs.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_Ts['+str(j)+'].arr_s['+str(i)+']')]; arrTemp_s_Ts.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_Ts['+str(j)+'].arr_T['+str(i)+']')]; arrTemp_T_Ts.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_us['+str(j)+'].arr_s['+str(i)+']')]; arrTemp_s_us.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_us['+str(j)+'].arr_u['+str(i)+']')]; arrTemp_u_us.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_pv['+str(j)+'].arr_v['+str(i)+']')]; arrTemp_v_pv.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_pv['+str(j)+'].arr_p['+str(i)+']')]; arrTemp_p_pv.append(val)
        
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_pVol['+str(j)+'].arr_V['+str(i)+']')]; arrTemp_V_pVol.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_pVol['+str(j)+'].arr_p['+str(i)+']')]; arrTemp_p_pVol.append(val)
    ##### end for #####
    
    s_curve_hs[j-1]=arrTemp_s_hs; arrTemp_s_hs=[]
    h_curve_hs[j-1]=arrTemp_h_hs; arrTemp_h_hs=[]
    s_curve_Ts[j-1]=arrTemp_s_Ts; arrTemp_s_Ts=[]
    T_curve_Ts[j-1]=arrTemp_T_Ts; arrTemp_T_Ts=[]
    s_curve_us[j-1]=arrTemp_s_us; arrTemp_s_us=[]
    u_curve_us[j-1]=arrTemp_u_us; arrTemp_u_us=[]
    v_curve_pv[j-1]=arrTemp_v_pv; arrTemp_v_pv=[]
    p_curve_pv[j-1]=arrTemp_p_pv; arrTemp_p_pv=[]
    
    V_curve_pVol[j-1]=arrTemp_V_pVol; arrTemp_V_pVol=[]
    p_curve_pVol[j-1]=arrTemp_p_pVol; arrTemp_p_pVol=[]
##### end for #####
print('------------------------------')

In [ ]:
fig.append(plt.figure(figsize=(20,20)))
for i in range(4):
    if((i==1-1)or(i==3-1)):
        plt.plot(V_curve_pVol[i], p_curve_pVol[i], linestyle='dashed')
##### end for #####

plt.grid()
plt.title('p vs. V'+ '; at time:' + str(tgtTime) + '[s]')
plt.xlabel('volume [m3]')
plt.ylabel('pressure [Pa]')
for i in range(0, 8-1):
    if((i==1)or(i==3)or(i==4)or(i==6)):
        root=[arr_V[i+1],arr_p[i+1]]
        head=[arr_V[i],arr_p[i]]
        plt.annotate('', xy=root, xytext=head,
                    arrowprops=dict(shrink=0, width=0.25, headwidth=8, headlength=14, connectionstyle='arc3',
                                    facecolor='black', edgecolor='black', linestyle='dashed')
                    )
    ##### end if #####
    
    if((i==5)or(i==7)):
        root=[arr_V[i+1],arr_p[i+1]]
        head=[arr_V[i],arr_p[i]]
        plt.annotate('', xy=root, xytext=head,
                    arrowprops=dict(shrink=0, width=0.25, headwidth=6, headlength=6, connectionstyle='arc3',
                                    facecolor='black', edgecolor='black', linestyle='dashed')
                    )
    ##### end if #####
##### end for #####

plt.scatter(x=arr_V, y=arr_p, color='red', s=20)

'''or(i==3)or(i==4)or(i==5)or(i==6)or(i==7)'''
print('------------------------------')

In [ ]:
i_df=1
tgtTime=70
widthTime=1

#******************************
#*** find the row of target time ***
rowTgt=0
for i in range(len(df[i_df])):
    if((tgtTime-widthTime <= df[i_df].at[i,'time']) and (df[i_df].at[i,'time'] <= tgtTime) ):
        rowTgt=i
        timeExtrd=df[i_df].at[i,'time']
    ##### end if #####
##### end for #####

print("row extracted = "+str(rowTgt))
print("time of extracted = "+str(timeExtrd))

arr_h=[]
arr_s=[]
arr_v=[]
arr_T=[]
arr_u=[]
for i in range(1,5+1):
    # ***** extract specified variables of target row *****
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_h['+str(i)+']')]
    arr_h.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_s['+str(i)+']')]
    arr_s.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_v['+str(i)+']')]
    arr_v.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_T['+str(i)+']')]
    arr_T.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.arr_u['+str(i)+']')]
    arr_u.append(val)
##### end for #####

arr_V=[]
arr_p=[]
for i in range(1,8+1):
    # ***** extract specified variables of target row *****
    val= df[i_df].at[rowTgt, ('PistonCylinder.arr_p['+str(i)+']')]
    arr_p.append(val)
    val= df[i_df].at[rowTgt, ('PistonCylinder.arr_V['+str(i)+']')]
    arr_V.append(val)
##### end for #####

nPts= df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_hs[1].nPts')]
s_curve_hs=[[]]*4; h_curve_hs=[[]]*4
v_curve_pv=[[]]*4; p_curve_pv=[[]]*4
s_curve_Ts=[[]]*4; T_curve_Ts=[[]]*4
s_curve_us=[[]]*4; u_curve_us=[[]]*4
V_curve_pVol=[[]]*4; p_curve_pVol=[[]]*4

arrTemp_s_hs=[]; arrTemp_h_hs=[]
arrTemp_s_Ts=[]; arrTemp_T_Ts=[]
arrTemp_s_us=[]; arrTemp_u_us=[]
arrTemp_v_pv=[]; arrTemp_p_pv=[]
arrTemp_V_pVol=[]; arrTemp_p_pVol=[]

for j in range(1, 4+1):
    for i in range(1, nPts+1):
        # ***** extract specified variables of target row *****
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_hs['+str(j)+'].arr_s['+str(i)+']')]; arrTemp_s_hs.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_hs['+str(j)+'].arr_h['+str(i)+']')]; arrTemp_h_hs.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_Ts['+str(j)+'].arr_s['+str(i)+']')]; arrTemp_s_Ts.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_Ts['+str(j)+'].arr_T['+str(i)+']')]; arrTemp_T_Ts.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_us['+str(j)+'].arr_s['+str(i)+']')]; arrTemp_s_us.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_us['+str(j)+'].arr_u['+str(i)+']')]; arrTemp_u_us.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_pv['+str(j)+'].arr_v['+str(i)+']')]; arrTemp_v_pv.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_pv['+str(j)+'].arr_p['+str(i)+']')]; arrTemp_p_pv.append(val)
        
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_pVol['+str(j)+'].arr_V['+str(i)+']')]; arrTemp_V_pVol.append(val)
        val=df[i_df].at[rowTgt, ('PistonCylinder.OttoCycle.curve_pVol['+str(j)+'].arr_p['+str(i)+']')]; arrTemp_p_pVol.append(val)
    ##### end for #####
    
    s_curve_hs[j-1]=arrTemp_s_hs; arrTemp_s_hs=[]
    h_curve_hs[j-1]=arrTemp_h_hs; arrTemp_h_hs=[]
    s_curve_Ts[j-1]=arrTemp_s_Ts; arrTemp_s_Ts=[]
    T_curve_Ts[j-1]=arrTemp_T_Ts; arrTemp_T_Ts=[]
    s_curve_us[j-1]=arrTemp_s_us; arrTemp_s_us=[]
    u_curve_us[j-1]=arrTemp_u_us; arrTemp_u_us=[]
    v_curve_pv[j-1]=arrTemp_v_pv; arrTemp_v_pv=[]
    p_curve_pv[j-1]=arrTemp_p_pv; arrTemp_p_pv=[]
    
    V_curve_pVol[j-1]=arrTemp_V_pVol; arrTemp_V_pVol=[]
    p_curve_pVol[j-1]=arrTemp_p_pVol; arrTemp_p_pVol=[]
##### end for #####
print('------------------------------')

In [ ]:
fig.append(plt.figure(figsize=(20,20)))
for i in range(4):
    if((i==1-1)or(i==3-1)):
        plt.plot(V_curve_pVol[i], p_curve_pVol[i], linestyle='dashed')
##### end for #####

plt.grid()
plt.title('p vs. V'+ '; at time:' + str(tgtTime) + '[s]')
plt.xlabel('volume [m3]')
plt.ylabel('pressure [Pa]')
for i in range(0, 8-1):
    if((i==1)or(i==3)or(i==4)or(i==6)):
        root=[arr_V[i+1],arr_p[i+1]]
        head=[arr_V[i],arr_p[i]]
        plt.annotate('', xy=root, xytext=head,
                    arrowprops=dict(shrink=0, width=0.25, headwidth=6, headlength=14, connectionstyle='arc3',
                                    facecolor='black', edgecolor='black', linestyle='dashed')
                    )
    ##### end if #####
    
    if((i==5)or(i==7)):
        root=[arr_V[i+1],arr_p[i+1]]
        head=[arr_V[i],arr_p[i]]
        plt.annotate('', xy=root, xytext=head,
                    arrowprops=dict(shrink=0, width=0.25, headwidth=6, headlength=6, connectionstyle='arc3',
                                    facecolor='black', edgecolor='black', linestyle='dashed')
                    )
    ##### end if #####
##### end for #####

plt.scatter(x=arr_V, y=arr_p, color='red', s=20)

print('------------------------------')

In [ ]:
fig.append(plt.figure(figsize=(10,10)))
plt.scatter(x=arr_s, y=arr_h, color='red', s=60)

for i in range(4):
    plt.plot(s_curve_hs[i], h_curve_hs[i], linestyle='dashed')
##### end for #####
plt.grid()
plt.title('h vs. s' + '; at time:' + str(tgtTime) + '[s]')
plt.xlabel('spec. entropy [J/(kg*K)]')
plt.ylabel('spec. enthalpy [J/kg]')
for i in range(0, 4):
    if((i==1-1)or(i==3-1)):
        plt.annotate('', xy=[arr_s[i+1],arr_h[i+1]], xytext=[arr_s[i],arr_h[i]],
                    arrowprops=dict(shrink=0, width=0.5, headwidth=8, headlength=14, connectionstyle='arc3',
                                    facecolor='gray', edgecolor='gray', linestyle='dashed')
                    )
##### end for #####
''''''
print('------------------------------')

In [ ]:
fig.append(plt.figure(figsize=(10,10)))
plt.scatter(x=arr_s, y=arr_u, color='red', s=60)

for i in range(4):
    plt.plot(s_curve_us[i], u_curve_us[i], linestyle='dashed')
##### end for #####
plt.grid()
plt.title('u vs. s' + '; at time:' + str(tgtTime) + '[s]')
plt.xlabel('spec. entropy [J/(kg*K)]')
plt.ylabel('spec. internal energy [J/kg]')

for i in range(0, 4):
    if((i==1-1)or(i==3-1)):
        root=[arr_s[i+1],arr_u[i+1]]
        head=[arr_s[i],arr_u[i]]
        plt.annotate('', xy=root, xytext=head,
                    arrowprops=dict(shrink=0, width=0.5, headwidth=8, headlength=14, connectionstyle='arc3',
                                    facecolor='gray', edgecolor='gray', linestyle='dashed')
                    )
##### end for #####
''''''
print('------------------------------')

In [ ]:
fig.append(plt.figure(figsize=(10,10)))
plt.scatter(x=arr_s, y=arr_T, color='red', s=60)
for i in range(4):
    plt.plot(s_curve_Ts[i], T_curve_Ts[i], linestyle='dashed')
##### end for #####
plt.grid()
plt.title('T vs. s'+ '; at time:' + str(tgtTime) + '[s]')
plt.xlabel('spec. entropy [J/(kg*K)]')
plt.ylabel('Temperature [K]')

for i in range(0, 4):
    if((i==1-1)or(i==3-1)):
        root=[arr_s[i+1],arr_T[i+1]]
        head=[arr_s[i],arr_T[i]]
        plt.annotate('', xy=root, xytext=head,
                    arrowprops=dict(shrink=0, width=0.5, headwidth=8, headlength=12, connectionstyle='arc3',
                                    facecolor='gray', edgecolor='gray', linestyle='dashed')
                    )
##### end for #####
''''''
print('------------------------------')

<h1>Process at the end of note</h1>

In [ ]:
print(""); print('------------------------------')
os.chdir(pathNotebook)
print("move to directory of notebook")
print("currend dir, back to: "+os.getcwd())
print('------------------------------')